In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy.odr as odr
import os
import sys
import arrow
import numpy as np
import pandas as pd
from math import pi
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
from datetime import datetime
from numpy import nan_to_num
from matplotlib.colors import LogNorm
from matplotlib.ticker import ScalarFormatter
import os
import sys
import arrow
import numpy as np
import pandas as pd
from math import pi
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
from datetime import date
from numpy import nan_to_num
from matplotlib.colors import LogNorm
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from datetime import datetime, timedelta
import glob

In [2]:
# Path definitions
RAW_PATH = rf"C:\Users\haika\Desktop\May_Research\may_datasets\binned_data"
OUTPUT_PATH = rf"C:\Users\haika\Desktop\May_Research\may_datasets\wavelength_corrected_data"
WAVELENGTH_INFO_PATH = rf"C:\Users\haika\Desktop\May_Research\may_datasets\wavelength_info_list"

# Define target wavelengths (you may need to adjust these)
# Format: [blue, green, red] as per your function comments
SC_NEW_WAVELENGTHS = [450, 550, 700]    # Target scattering wavelengths
ABS_NEW_WAVELENGTHS = [470, 532, 660]   # Target absorption wavelengths

In [3]:
def SAE_fit_one_measurement( wavelength, SAE_fit, SAE_constant):
    return  SAE_constant*np.power(wavelength, -SAE_fit)

def AAE_fit_one_measurement( wavelength, AAE_fit, AAE_constant):
    return  AAE_constant*np.power(wavelength, -AAE_fit)

#IMPORTANT: sc_ori_wavelengths = [lowest, mid, highest] = [w3, w2, w1] = [blue, green, red]
#IMPORTANT: abs_ori_wavelengths = [lowest, mid, highest] = [w3, w2, w1] = [blue, green, red]
def change_wavelength(df, sc_column_names, abs_column_names, sc_ori_wavelengths, abs_ori_wavelengths, sc_new_wavelengths, abs_new_wavelengths):
    
    """
    THIS PART OF CODE FOCUSES ON SCATTERING COEFFICIENTS
    """

    #Only changes the scattering coefficients if the original wavelengths are different from the new wavelengths
    if(sc_ori_wavelengths[0] != sc_new_wavelengths[0] and 
       sc_ori_wavelengths[1] != sc_new_wavelengths[1] and 
       sc_ori_wavelengths[2] != sc_new_wavelengths[2]):
        #Read in original wavelengths for SC
        w1 = sc_ori_wavelengths[2]  #red
        w2 = sc_ori_wavelengths[1]  #green
        w3 = sc_ori_wavelengths[0]  #blue

        #Read in original wavelength column values for SC
        bscat_wvl_red = df[sc_column_names[2]] #red
        bscat_wvl_green = df[sc_column_names[1]] #green
        bscat_wvl_blue = df[sc_column_names[0]] #blue

        if(w1 <= 0 and w2 <= 0 and w3 <= 0):  #added by Cora
            print("WARNING: wavelength <= 0")

        #W1, W2, W3 will be in decreasing order or red, green, blue
        wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

        temp_SAE = np.nan; temp_SAE_constant = np.nan      # Fitting parameters
        temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty in fit parameters

        SAE_three_lambda = np.empty(len(bscat_wvl_red)); SAE_constant = np.empty(len(bscat_wvl_red))
        sigma_SAE = np.empty(len(bscat_wvl_red)); sigma_constant = np.empty(len(bscat_wvl_red))

        #hide invalid values  #added by cora
        #bscat_wvl_red[bscat_wvl_red ==0]
        mask_red = bscat_wvl_red.mask(bscat_wvl_red == 0)  # 0 replaced by NaN
        mask_green = bscat_wvl_green.mask(bscat_wvl_green == 0)
        mask_blue = bscat_wvl_blue.mask(bscat_wvl_blue == 0)

        mask_RG = (mask_red/mask_green).mask((mask_red/mask_green) == 1)
        mask_RG = mask_RG.mask((mask_red/mask_green) < 0)

        mask_RB = (mask_red/mask_blue).mask((mask_red/mask_blue) == 1)
        mask_RB = mask_RB.mask((mask_red/mask_blue) < 0)

        mask_GB = (mask_green/mask_blue).mask((mask_green/mask_blue) == 1)
        mask_GB = mask_GB.mask((mask_green/mask_blue) < 0)

        #mask_red.plot(kind='line')  #Check plot

        #compute SAE_two_lambda, which are used later when initializing SAE_three_lambda
        SAE_R_G = -np.log(mask_RG)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
        SAE_R_B = -np.log(mask_RB)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
        SAE_G_B = -np.log(mask_GB)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

        W_coef = np.zeros(np.size((SAE_G_B.values)))
        w_sigma = np.zeros(np.size((SAE_G_B.values)))

        for m in range(len(bscat_wvl_red)):
                if (np.isnan(bscat_wvl_red[m])==0 and np.isnan(bscat_wvl_green[m])==0 and np.isnan(bscat_wvl_blue[m])==0):

                    one_babs_at_three_lambda=[bscat_wvl_red[m],bscat_wvl_green[m],bscat_wvl_blue[m]]
                    
                    #use SAE_G_B as initial input 
                    if((np.power(wavelength_instrument[1], -SAE_G_B.values[m])) > 0):
                        W_coef = np.array([SAE_G_B.values[m], bscat_wvl_green[m]/ (np.power(wavelength_instrument[1], -SAE_G_B.values[m]))] )
                        # first part before coma is SAE_fit, second part after coma is SAE_Constat or C
                        try:
                            W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                            w_sigma = np.sqrt(np.diag(w_sigma))
                        except RuntimeError:
                            pass
                    temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                    #use SAE_R_G as initial input
                    if(((np.power(wavelength_instrument[0], -SAE_R_G.values[m]))) > 0 ):
                        W_coef = np.array([SAE_R_G.values[m], bscat_wvl_red[m]/ (np.power(wavelength_instrument[0], -SAE_R_G.values[m]))]  ) 
                        try:          
                            W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                            w_sigma = np.sqrt(np.diag(w_sigma))
                        except RuntimeError:
                            pass    #do not replace the previous results

                    #make sure w_sigma exists
                    if(w_sigma[0] != -1):  # Uzzal don't know why Cora added this?
                        #check if the current inital guess (SAE_R_G) is better than the previous one (SAE_G_B): ? w_sigma is smaller
                        if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                            temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                        else:
                            pass       #do not replace the previous results
                
                    #use SAE_R_B as initial input
                    if((np.power(wavelength_instrument[2], -SAE_R_B.values[m])) > 0):
                        W_coef = np.array([SAE_R_B.values[m],bscat_wvl_blue[m]/ (np.power(wavelength_instrument[2], -SAE_R_B.values[m]))])
                        try:   
                            W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                            w_sigma = np.sqrt(np.diag(w_sigma))
                        except RuntimeError:
                            pass
                    #make sure w_sigma exists
                    if(w_sigma[0] != -1):               
                        #check if the current initial guess (SAE_R_B) is better than the previous one (SAE_G_B or SAE_R_G): ? w_sigma is smaller
                        if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                            temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                        else: 
                            pass #do not replace the previous results   
                
                #Output the results
                SAE_three_lambda[m]=temp_SAE; SAE_constant[m]=temp_SAE_constant
                sigma_SAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
                #clear temp_variables
                temp_SAE=np.nan;temp_SAE_constant=np.nan
                temp_sigma=np.nan;temp_sigma_constant=np.nan

        b_scat_red=SAE_constant*(np.power(sc_new_wavelengths[2], -SAE_three_lambda))
        b_scat_green=SAE_constant*(np.power(sc_new_wavelengths[1], -SAE_three_lambda))
        b_scat_blue=SAE_constant*(np.power(sc_new_wavelengths[0], -SAE_three_lambda))

        #Replace the original scattering coefficients with the new ones
        df[sc_column_names[2]] = b_scat_red
        df[sc_column_names[1]] = b_scat_green
        df[sc_column_names[0]] = b_scat_blue

    """
    THIS PART OF CODE FOCUSES ON ABSORPTION COEFFICIENTS
    """

    #Only changes the absoprtion coefficients if the original wavelengths are different from the new wavelengths
    if(abs_ori_wavelengths[0] != abs_new_wavelengths[0] and 
       abs_ori_wavelengths[1] != abs_new_wavelengths[1] and 
       abs_ori_wavelengths[2] != abs_new_wavelengths[2]):
        
        w1 = abs_ori_wavelengths[2]  #red
        w2 = abs_ori_wavelengths[1]  #green
        w3 = abs_ori_wavelengths[0]  #blue

        #Read in original wavelength column values for SC
        babs_wvl_red = df[abs_column_names[2]]#red
        babs_wvl_green = df[abs_column_names[1]]  #green
        babs_wvl_blue = df[abs_column_names[0]]#blue
    
        if(w1 <= 0 and w2 <= 0 and w3 <= 0):
            print("WARNING: wavelength <= 0")

        wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

        temp_AAE = np.nan; temp_AAE_constant = np.nan
        temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty of fit parameters

        AAE_three_lambda = np.empty(len(babs_wvl_red)); AAE_constant = np.empty(len(babs_wvl_red))
        sigma_AAE = np.empty(len(babs_wvl_red)); sigma_constant = np.empty(len(babs_wvl_red))

        #hide invalid values
        mask_red = babs_wvl_red.mask(babs_wvl_red == 0)
        mask_green = babs_wvl_green.mask(babs_wvl_green == 0)
        mask_blue = babs_wvl_blue.mask(babs_wvl_blue == 0)

        mask_RG = (mask_red/mask_green).mask((mask_red/mask_green == 1))
        mask_RG = mask_RG.mask((mask_red/mask_green) < 0)

        mask_RB = (mask_red/mask_blue).mask((mask_red/mask_blue) == 1)
        mask_RB = mask_RB.mask((mask_red/mask_blue) < 0)

        mask_GB = (mask_green/mask_blue).mask((mask_green/mask_blue) == 1)
        mask_GB = mask_GB.mask((mask_green/mask_blue) < 0)

        #compute AAE_two_lambda, which are used later when initializing AAE_three_lambda
        AAE_R_G = -np.log(mask_RG)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
        AAE_R_B = -np.log(mask_RB)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
        AAE_G_B = -np.log(mask_GB)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

        for m in range(len(babs_wvl_red)):
            if (np.isnan(babs_wvl_red[m])==0 and np.isnan(babs_wvl_green[m])==0 and np.isnan(babs_wvl_blue[m])==0):

                one_babs_at_three_lambda=[babs_wvl_red[m],babs_wvl_green[m],babs_wvl_blue[m]]
                    
                #use AAE_G_B as initial input
                W_coef = np.array([AAE_G_B.values[m], babs_wvl_green[m]/ (np.power(wavelength_instrument[1], -AAE_G_B.values[m]))] )
                try:
                    W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                    w_sigma = np.sqrt(np.diag(w_sigma))
                except RuntimeError:
                    pass
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                #use AAE_R_G as initial input
                W_coef = np.array([AAE_R_G.values[m], babs_wvl_red[m]/ (np.power(wavelength_instrument[0], -AAE_R_G.values[m]))]  ) 
                try:          
                    W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                    w_sigma = np.sqrt(np.diag(w_sigma))
                except RuntimeError:
                    pass
                
                #check if the current inital guess (AAE_R_G) is better than the previous one (AAE_G_B): ? w_sigma is smaller
                if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                    temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                else:
                    pass       #do not replace the previous results
                
                #use AAE_R_B as initial input
                W_coef = np.array([AAE_R_B.values[m],babs_wvl_blue[m]/ (np.power(wavelength_instrument[2], -AAE_R_B.values[m]))])
                try:   
                    W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                    w_sigma = np.sqrt(np.diag(w_sigma))
                except RuntimeError:
                    pass               
                #check if the current initial guess (AAE_R_B) is better than the previous one (AAE_G_B or AAE_R_G): ? w_sigma is smaller
                if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                    temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                else: 
                    pass #do not replace the previous results   
                
            #Output the results
            AAE_three_lambda[m]=temp_AAE; AAE_constant[m]=temp_AAE_constant
            sigma_AAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
            #clear temp_variables
            temp_AAE=np.nan;temp_AAE_constant=np.nan
            temp_sigma=np.nan;temp_sigma_constant=np.nan

        b_abs_red=AAE_constant*(np.power(abs_new_wavelengths[2], -AAE_three_lambda))
        b_abs_green=AAE_constant*(np.power(abs_new_wavelengths[1], -AAE_three_lambda))
        b_abs_blue=AAE_constant*(np.power(abs_new_wavelengths[0], -AAE_three_lambda))

        df[abs_column_names[2]] = b_abs_red
        df[abs_column_names[1]] = b_abs_green
        df[abs_column_names[0]] = b_abs_blue

    #Returns final df with corrected scattering and absorption coefficients
    return df



        


In [4]:
def read_wavelength_info_files():
    """
    Read all wavelength info CSV files and return a dictionary with campaign names as keys
    and wavelength information as values.
    """
    wavelength_info_dict = {}
    
    # Get all CSV files in the wavelength info directory
    wavelength_info_files = glob.glob(os.path.join(WAVELENGTH_INFO_PATH, "*.csv"))
    
    for file_path in wavelength_info_files:
        # Extract campaign name from filename
        campaign_name = os.path.splitext(os.path.basename(file_path))[0]
        
        try:
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Store the dataframe in the dictionary
            wavelength_info_dict[campaign_name] = df
            
            print(f"Loaded wavelength info for campaign: {campaign_name}")
            
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    return wavelength_info_dict

def parse_wavelength_info(wavelength_df):
    """
    Parse wavelength info DataFrame to extract column names and wavelengths
    for scattering and absorption coefficients.
    """
    # Separate scattering and absorption rows
    sc_rows = wavelength_df[wavelength_df['Column'].str.startswith('Sc')]
    abs_rows = wavelength_df[wavelength_df['Column'].str.startswith('Abs')]
    
    # Sort by wavelength to ensure [blue, green, red] order
    sc_rows = sc_rows.sort_values('ori_wavelength')
    abs_rows = abs_rows.sort_values('ori_wavelength')
    
    # Extract column names and wavelengths
    sc_column_names = sc_rows['Column'].tolist()
    sc_ori_wavelengths = sc_rows['ori_wavelength'].tolist()
    
    abs_column_names = abs_rows['Column'].tolist()
    abs_ori_wavelengths = abs_rows['ori_wavelength'].tolist()
    
    return sc_column_names, abs_column_names, sc_ori_wavelengths, abs_ori_wavelengths

def process_wavelength_corrections():
    """
    Process all campaign data files and apply wavelength corrections.
    """
    print("="*60)
    print("PROCESSING WAVELENGTH CORRECTIONS:")
    print("="*60)
    
    # Create output directory if it doesn't exist
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    
    # Read wavelength info files
    wavelength_info_dict = read_wavelength_info_files()
    
    # Process each campaign
    for campaign_name, wavelength_df in wavelength_info_dict.items():
        print(f"\nProcessing campaign: {campaign_name}")
        
        # Look for any file containing the campaign name
        campaign_files = glob.glob(os.path.join(RAW_PATH, f"*{campaign_name}*"))

        if not campaign_files:
            print(f"  No data files found containing: {campaign_name}")
            continue

        # Use the first matching file
        input_file = campaign_files[0]
        filename = os.path.basename(input_file)
        print(f"  Found data file: {filename}")
        
        try:
            # Read the campaign data
            df = pd.read_csv(input_file)
            print(f"  Loaded data with {len(df)} rows and {len(df.columns)} columns")
            
            # Parse wavelength information
            sc_column_names, abs_column_names, sc_ori_wavelengths, abs_ori_wavelengths = parse_wavelength_info(wavelength_df)
            
            print(f"  Scattering columns: {sc_column_names}")
            print(f"  Scattering wavelengths: {sc_ori_wavelengths} nm")
            print(f"  Absorption columns: {abs_column_names}")
            print(f"  Absorption wavelengths: {abs_ori_wavelengths} nm")
            
            # Check if columns exist in the dataframe
            missing_columns = []
            for col in sc_column_names + abs_column_names:
                if col not in df.columns:
                    missing_columns.append(col)
            
            if missing_columns:
                print(f"  Warning: Missing columns in data: {missing_columns}")
                # You may want to skip this campaign or handle missing columns
                continue
            
            # Apply wavelength corrections
            print(f"  Applying wavelength corrections...")
            print(f"    Target scattering wavelengths: {SC_NEW_WAVELENGTHS} nm")
            print(f"    Target absorption wavelengths: {ABS_NEW_WAVELENGTHS} nm")
            
            corrected_df = change_wavelength(
                df=df,
                sc_column_names=sc_column_names,
                abs_column_names=abs_column_names,
                sc_ori_wavelengths=sc_ori_wavelengths,
                abs_ori_wavelengths=abs_ori_wavelengths,
                sc_new_wavelengths=SC_NEW_WAVELENGTHS,
                abs_new_wavelengths=ABS_NEW_WAVELENGTHS
            )
            
            # Save the wavelength-corrected data
            output_filename = f"{campaign_name}_wavelengthcorrected.csv"
            output_path = os.path.join(OUTPUT_PATH, output_filename)
            corrected_df.to_csv(output_path, index=False)
            
            print(f"  Saved wavelength-corrected data to: {output_filename}")
            
        except Exception as e:
            print(f"  Error processing {campaign_name}: {e}")
            import traceback
            traceback.print_exc()

def display_wavelength_info():
    """
    Display wavelength information for all campaigns for verification.
    """
    print("\n" + "="*60)
    print("WAVELENGTH INFO SUMMARY:")
    print("="*60)
    
    wavelength_info_dict = read_wavelength_info_files()
    
    for campaign_name, wavelength_df in wavelength_info_dict.items():
        print(f"\nCampaign: {campaign_name}")
        print("-" * 40)
        
        sc_column_names, abs_column_names, sc_ori_wavelengths, abs_ori_wavelengths = parse_wavelength_info(wavelength_df)
        
        print("Scattering:")
        for col, wl in zip(sc_column_names, sc_ori_wavelengths):
            print(f"  {col}: {wl} nm")
        
        print("Absorption:")
        for col, wl in zip(abs_column_names, abs_ori_wavelengths):
            print(f"  {col}: {wl} nm")

In [6]:
#Main run
if __name__ == "__main__":
    # First display the wavelength info for verification
    display_wavelength_info()
    
    # Then process the wavelength corrections
    process_wavelength_corrections()
    
    print("\n" + "="*60)
    print("WAVELENGTH CORRECTION PROCESSING COMPLETE!")
    print("="*60)



WAVELENGTH INFO SUMMARY:
Loaded wavelength info for campaign: ACMEV
Loaded wavelength info for campaign: BBOP
Loaded wavelength info for campaign: CACTI
Loaded wavelength info for campaign: CARES
Loaded wavelength info for campaign: GOAMAZON
Loaded wavelength info for campaign: ISDAC
Loaded wavelength info for campaign: TCAP2012
Loaded wavelength info for campaign: TCAP2013

Campaign: ACMEV
----------------------------------------
Scattering:
  Sc450_total: 450 nm
  Sc550_total: 550 nm
  Sc700_total: 700 nm
Absorption:
  Abs470_total: 462 nm
  Abs532_total: 523 nm
  Abs660_total: 648 nm

Campaign: BBOP
----------------------------------------
Scattering:
  Sc450_total: 450 nm
  Sc550_total: 550 nm
  Sc700_total: 700 nm
Absorption:
  Abs470_total: 462 nm
  Abs532_total: 523 nm
  Abs660_total: 648 nm

Campaign: CACTI
----------------------------------------
Scattering:
  Sc450_total: 450 nm
  Sc550_total: 550 nm
  Sc700_total: 700 nm
Absorption:
  Abs470_total: 464 nm
  Abs532_total: 52

C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:186: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:194: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:208: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:5: RuntimeWarning: overflow encountered in power
  return  AAE_constant*np.power(wavelength, -AAE_fit)
c:\Users\haika\AppData\Local\Programs\Python\P

  Saved wavelength-corrected data to: ACMEV_wavelengthcorrected.csv

Processing campaign: BBOP
  Found data file: BBOP_binned.csv
  Loaded data with 64847 rows and 50 columns
  Scattering columns: ['Sc450_total', 'Sc550_total', 'Sc700_total']
  Scattering wavelengths: [450, 550, 700] nm
  Absorption columns: ['Abs470_total', 'Abs532_total', 'Abs660_total']
  Absorption wavelengths: [462, 523, 648] nm
  Applying wavelength corrections...
    Target scattering wavelengths: [450, 550, 700] nm
    Target absorption wavelengths: [470, 532, 660] nm


C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:194: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:208: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:186: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)


  Saved wavelength-corrected data to: BBOP_wavelengthcorrected.csv

Processing campaign: CACTI
  Found data file: CACTI_binned.csv
  Loaded data with 38596 rows and 50 columns
  Scattering columns: ['Sc450_total', 'Sc550_total', 'Sc700_total']
  Scattering wavelengths: [450, 550, 700] nm
  Absorption columns: ['Abs470_total', 'Abs532_total', 'Abs660_total']
  Absorption wavelengths: [464, 529, 648] nm
  Applying wavelength corrections...
    Target scattering wavelengths: [450, 550, 700] nm
    Target absorption wavelengths: [470, 532, 660] nm


...

  Saved wavelength-corrected data to: ISDAC_wavelengthcorrected.csv

Processing campaign: TCAP2012
  Found data file: TCAP2012_binned.csv
  Loaded data with 15523 rows and 50 columns
  Scattering columns: ['Sc450_total', 'Sc550_total', 'Sc700_total']
  Scattering wavelengths: [450, 550, 700] nm
  Absorption columns: ['Abs470_total', 'Abs532_total', 'Abs660_total']
  Absorption wavelengths: [462, 523, 648] nm
  Applying wavelength corrections...
    Target scattering wavelengths: [450, 550, 700] nm
    Target absorption wavelengths: [470, 532, 660] nm


C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:186: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:194: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:208: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:184: RuntimeWarning: overflow encountered in power
  W_coef = np.array([AAE_G_B.values[m], babs_wvl_green[m]/ (np.power(wavelength_instrument[1], -AA

  Saved wavelength-corrected data to: TCAP2012_wavelengthcorrected.csv

Processing campaign: TCAP2013
  Found data file: TCAP2013_binned.csv
  Loaded data with 18901 rows and 50 columns
  Scattering columns: ['Sc450_total', 'Sc550_total', 'Sc700_total']
  Scattering wavelengths: [450, 550, 700] nm
  Absorption columns: ['Abs470_total', 'Abs532_total', 'Abs660_total']
  Absorption wavelengths: [462, 523, 648] nm
  Applying wavelength corrections...
    Target scattering wavelengths: [450, 550, 700] nm
    Target absorption wavelengths: [470, 532, 660] nm


C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:186: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:194: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:208: OptimizeWarning: Covariance of the parameters could not be estimated
  W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
C:\Users\haika\AppData\Local\Temp\ipykernel_6328\453055719.py:5: RuntimeWarning: overflow encountered in power
  return  AAE_constant*np.power(wavelength, -AAE_fit)
C:\Users\haika\AppData\Local\Temp\ipykernel_63

  Saved wavelength-corrected data to: TCAP2013_wavelengthcorrected.csv

WAVELENGTH CORRECTION PROCESSING COMPLETE!
